## Maze

Als *state* betrachten wir eine Position, d.h. ein Tuple (x,y).

In [2]:
%%writefile maze1.txt
##########
#   #    #
#   ##   #
# #Z#  ###
# ## #   #
#   S #  #
#     #  #
#        #
##########

Overwriting maze1.txt


In [3]:
f = open('maze1.txt')       
grid = f.read().splitlines()
f.close()
height = len(grid)
width = len(grid[0])
for x in range(height):
    for y in range(width):
        if grid[x][y] == 'S':
            start = (x,y)
        elif grid[x][y] == 'Z':
            ziel = (x,y)

print(f'Höhe = {height}, Breite = {width}')
print(f'{start=}, {ziel=}')

dirs = [(0,-1),(0,1),(-1,0),(1,0)]    


Höhe = 9, Breite = 10
start=(5, 4), ziel=(3, 3)


### BFS - Breitensuche

Für den konkreten Anwendungsfall müssen die Funktionen *nextstates* und *goaltest* implementiert werden.

In [4]:
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def nextstates(state):
    '''
    returns:  Liste mit möglichen Folgestellungen zu state
    '''
    x, y = state
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp    
    
def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel


# Aufruf:
prev, goal = bfs(start)
path = reconstructPath(prev,goal) 
print(path)

[(5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (3, 3)]


In [5]:
print(getMoves(path))

LLLUUURRD


### AStar

In [37]:
from heapq import heappop, heappush
def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # die Rückwärtskosten
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            gg = g[state] + 1       
            if v not in prev or gg < g[v]:     
                g[v] = gg                     
                heappush(frontier,(g[v]+h(v),v))  
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path


def nextstates(state):
    '''
    returns:  Liste mit möglichen Folgestellungen zu state
    '''
    x, y = state
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp   

def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel

def h(state):
    '''
    returns: Vorwärtskosten laut Heuristik
    '''
    x, y = state
    x1, y1 = ziel
    return abs(x1-x)+abs(y1-y)

#Aufruf:
prev, goal = astar(start)
path = reconstructPath(prev,goal) 
print(getMoves(path))

LLLUUURRD


### Beamsearch

Beamsearch bietet sich an, wenn wir für AStar keine zulässige Heuristik finden, die in vernünftiger Zeit eine Lösung findet. Dann machen wir eine Breitensuche aber in jeder Ebene beschränken wir uns auf eine vorgegebene Anzahl von Möglichkeiten (beamwidth), die wir weiter betrachten. Bei der Bewertung der Möglichkeiten nutzen wir auch unzulässige Heuristiken. Eine optimale Lösung ist nicht garantiert, aber häufig werden recht gute Lösungen gefunden.

In [36]:
def beamsearch(s, beamwidth): 
    frontier =[s]
    prev = {s:None}
    nextfrontier = []
    while frontier:
        for state in frontier:
            if goaltest(state):
                return prev, state
            for v in nextstates(state):
                if v not in prev:
                    prev[v] = state
                    nextfrontier.append(v)
        
        frontier = sorted(nextfrontier,key=lambda x: h(x)) 
        frontier = frontier[:beamwidth]
        nextfrontier = []
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path


def nextstates(state):
    '''
    returns:  Liste mit möglichen Folgestellungen zu state
    '''
    x, y = state
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp   

def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel

def h(state):
    '''
    returns: Vorwärtskosten laut Heuristik
    '''
    x, y = state
    x1, y1 = ziel
    return abs(x1-x)+abs(y1-y)
    
#Aufruf:
BEAMWIDTH = 10
prev, goal = beamsearch(start,BEAMWIDTH)
path = reconstructPath(prev,goal) 
print(getMoves(path))

LLLUUURRD


### Pfad und Aktionen

Häufig will man nicht nur den Pfad ausgeben, sondern die Beschreibung der Aktionen, mit der dieser Pfad durchlaufen wird. Dazu muss die Funktion *getMove* implementiert werden.

In [1]:
def getMove(s1, s2):
    '''
    returns: die Beschreibung des Übergangs von state s1 zu state s2
    '''
    x1,y1 = s1
    x2,y2 = s2
    if x1 < x2: return 'D'   # down
    if x1 > x2: return 'U'   # up
    if y1 < y2: return 'R'   # right
    if y1 > y2: return 'L'   # left

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves

#Aufruf
#print(getMoves(path))